# Data retrieval using Pyrocko with Squirrel

With the addition of squirrel to pyrocko the way of requesting data has changed. How to do it with out the squirrel is shown in the notebook [pyrocko old](x_Pyrocko_old.ipynb). The advantage of squirrel is that it memorizes if the reqeusted data is already in your squirrel database, and therefore does not download it again, but just reloads it. 

[Reference](https://pyrocko.org/docs/current/library/examples/squirrel/cli_tool.html#variant-1-quick-and-dirty-script)


### Tabel of content:
- [Event](#Event-catalog)
- [Station](#Stations)
- [Waveform](#Waveform)

- [Summary](#Summary)

In [ ]:
import numpy as np
from pyrocko import progress
from pyrocko.util import str_to_time_fillup as stt
from pyrocko.squirrel import Squirrel

First we initialize a squirrel instance.

In [ ]:
sq = Squirrel()

## Event catalog
Here we have to give it a catalog name, at the moment *geofon* and *gcmt* are implemented.

In [ ]:
sq.add_catalog('geofon') # or gcmt

In [ ]:
tmin = stt('2016-10-30 00:00:00')
tmax = stt('2016-10-30 23:50:00')

sq.update(tmin=tmin, tmax=tmax)

With the *get_events()* function, we select now all events within the time span defined beforehand.

In [ ]:
evs = sq.get_events()

In [ ]:
len(evs)

We can iterate through the evs and for example select the specific time of an event. Here the Norcia earthquake.

In [ ]:
for ev in evs:
    if ev.magnitude > 6:
        print(ev.magnitude, ev.region)
        
        if ev.region == 'Central Italy' and ev.magnitude == 6.5:
            tevent = ev.time
            print(tevent)

## Stations

Again, we need to select a source, here *BGR*. Further, we can specify which network, stations and channels we are interested in. This limits further requests to stations that fit with the specification, meaning may be a restriction. 

In [ ]:
# Add online data source. Enable access to BGR's FDSN web service and restrict
# to GR network and LH channels only:
sq.add_fdsn('bgr', query_args=dict(network='GR', channel='LH?'))

Similar to the query option *level* squirrel has *get_stations()*, *get_channels()* or *get_response()*. All of them can be further restricted by using the *code* argument.

In [ ]:
stas = sq.get_stations(codes='*.*.*')
# stas = sq.get_stations(codes='*.BFO.*')
print(len(stas))

In [ ]:
for sta in stas:
    print(sta)

In [ ]:
chas = sq.get_channels(codes='*.BFO.*.*')
print(len(chas))

In [ ]:
for cha in chas:
    print(cha)
    print(cha)

In [ ]:
resps = sq.get_response(tmin=tmin, tmax=tmax, codes='GR.BFO..LHZ')
print(resps)

Note that for *get_response()* only one station at once is allowed. Therefore, to get all responses of your stations you have to use a loop. To can give the channel object directly, as it will automatically retrieves all informations.

In [ ]:
for cha in chas:
    resps = sq.get_response(cha)#.get_effective()
    print(cha)
    print(resps)

## Waveform

Here, we want to request the waveform for the Norcia earthquake, for one specific station. 

Before the download we need to update the waveform promises ([Ref](https://pyrocko.org/docs/current/library/reference/squirrel/base.html#pyrocko.squirrel.base.Squirrel.update_waveform_promises)).



In [ ]:
tmin = tevent + 0
tmax = tevent + 1500
# Ensure meta-data from online sources is up to date for the selected time
# span:
sq.update(tmin=tmin, tmax=tmax)

# Allow waveform download for station BFO. This does not download anything
# yet, it just enables downloads later, when needed. Omit `tmin`, `tmax`,
# and `codes` to enable download of everything selected in `add_fdsn(...)`.
sq.update_waveform_promises(tmin=tmin, tmax=tmax, codes='*.BFO.*.*')

With the *chopper_waveforms* function we download the waveform and store them.

In [ ]:
batches = sq.chopper_waveforms(
        tmin=tmin,
        tmax=tmax,
        codes='*.BFO.*.LH*',
        want_incomplete=False,   # Skip incomplete windows.
        snap_window=True)

In [ ]:
for batch in batches:
    for tr in batch.traces:        
        print(tr)        
        tr.plot()

Restitution of the waveform is also possible, in a rather easy fashion by combining it with *get_response()*.

In [ ]:
# Frequency band for restitution:
fmin = 0.01
fmax = 0.05

# Time length for padding (half of overlap).
tpad = 1.0 / fmin

batches = sq.chopper_waveforms(
        tmin=tmin,
        tmax=tmax,
        codes='*.BFO.*.LH*',
#         tinc=3600.,              # One hour time windows (payload).
#         tpad=tpad,               # Add padding to absorb processing artifacts.
        want_incomplete=False,   # Skip incomplete windows.
        snap_window=True)

for batch in batches:
    for tr in batch.traces:
        resp = sq.get_response(tr).get_effective()
        
        tr = tr.transfer(
            tpad,                  # Fade in / fade out length.
            (0.5*fmin, fmin,
             fmax, 2.0*fmax),      # Frequency taper.
            resp,                  # Complex frequency response of instrument.
            invert=True,           # Use inverse of instrument response.
            cut_off_fading=False)  # Disable internal trimming.
        
        tr.chop(batch.tmin, batch.tmax)
        
        print(tr)        
        tr.plot()


## Summary

You have learned how to use pyrocko and the squirrel to collect information about events, stations and waveforms.